# Import Modules

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from src import detect_faces, show_bboxes
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image

import torch
from torchvision import transforms, datasets
import numpy as np
import os

# Path Definitions

In [3]:
dataset_path = '../Dataset/emotiw/test/'
face_coordinates_directory = '../Dataset/FaceCoordinates/'
processed_face_ex = '../Dataset/CroppedFaces/test/'

# Loading Test Dataset

In [4]:
test_dataset = datasets.ImageFolder(dataset_path)

class_names = test_dataset.classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: '../Dataset/emotiw/test/'

In [ ]:
class_names

In [ ]:
test_filelist = sorted(os.listdir(dataset_path + 'test_shared/'))

In [ ]:
test_filelist = [x.split('.')[0] for x in test_filelist]

In [ ]:
print(test_filelist[:10])

In [ ]:
print(len(test_dataset))

# Crop Faces

In [ ]:
for i in range(len(test_dataset)):
    print(test_filelist[i])
    image, label = test_dataset[i]

    face_list = []
    landmarks_new_coordinates = []

    if os.path.isfile(processed_face_ex + 'test/' + test_filelist[i] + '.npz'):
        print(test_filelist[i] + ' Already present')
        continue

    bbox_lm = np.load(face_coordinates_directory + 'test/' + test_filelist[i] +'.npz')
    bounding_boxes = bbox_lm['a']
    if bounding_boxes.size == 0 or (bounding_boxes[0] == 0).all():
        print("No bounding boxes for " + test_filelist[i] + ". Adding empty file for the same")
        np.savez(processed_face_ex + test_filelist[i], a = np.zeros(1), b = np.zeros(1))
        continue
    landmarks = bbox_lm['b']

    for j in range(len(bounding_boxes)):
        bbox_coordinates = bounding_boxes[j]
        landmark = landmarks[j]
        img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))

        x = bbox_coordinates[0]
        y = bbox_coordinates[1]
        for k in range(5):
            landmark[k] -= x
            landmark[k+5] -= y
        img_face = np.array(img_face)
        landmark = np.array(landmark)


        if len(face_list) != 0:
            if img_face.shape[0] == face_list[-1].shape[0]:
                img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                img_face = np.array(img_face)
                landmark +=1

        face_list.append(img_face)
        landmarks_new_coordinates.append(landmark)
    face_list = np.asarray(face_list)
    landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
    np.savez(processed_face_ex + test_filelist[i], a = face_list, b = landmarks_new_coordinates)

    print(i)